Project 1  Rubric.   Boot Camp: Data Visualization and Data Analytics 
Question 4 Analysis
Source Data file: 'clean_states.csv', a subset of ‘USA Real Estate Datatest’ from Kaggle.com https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset

Represent the house prices by cities on a plot. Plot in a different color for each city, plot on a map with Latitude and Longtitude of each city either for just NY State. Could also map plot by cities of each State and have the color be different for each State. Scatter Plot each city with different tick / color for each city. 

4 selected states: Connecticut, Massachusetts, New Jersey, New York. 

Average Price and Standard Deviation (std) for each State reveals NY with the highest variance in STD, and Mean being houses with 3 Bedrooms and 2 Bathrooms.  

Assumptions: 1. clean data file may contain duplicates if not removed perhaps on criteria of Sold Date and Address. 
2. The df created an Unmaed:0 at the 0 index, these are the row numbers assigned to each record in the dataset. Do not want to use that for House ID. 

In [154]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
import os
import hvplot.pandas

In [155]:
%matplotlib widget

In [156]:
#read csv file
states_df=pd.read_csv("clean_states.csv")
states_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
0,27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
1,27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
2,27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
3,27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
4,27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...,...
185883,733133,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
185884,733136,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
185885,733146,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
185886,733147,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [157]:
#find the Average/ Mean of house prices, and the Standard Deviation. 
states_df.groupby("state").agg(["mean","std"]).round()

C:\Users\Owner\AppData\Local\Temp\ipykernel_2856\3925755002.py:2: FutureWarning: ['city'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  states_df.groupby("state").agg(["mean","std"]).round()


Unnamed: 0               price            bed      bath       \
                    mean       std      mean       std mean  std mean  std   
state                                                                        
Connecticut     366855.0  130123.0  326020.0  146810.0  3.0  1.0  2.0  1.0   
Massachusetts   142859.0   71148.0  645017.0  366150.0  3.0  1.0  2.0  1.0   
New Jersey      646053.0   63552.0  395038.0  202851.0  3.0  1.0  2.0  1.0   
New York        514894.0  104704.0  764468.0  442436.0  3.0  1.0  2.0  1.0   

              acre_lot      house_size         
                  mean  std       mean    std  
state                                          
Connecticut        1.0  1.0     1716.0  655.0  
Massachusetts      0.0  0.0     1860.0  839.0  
New Jersey         0.0  0.0     1760.0  668.0  
New York           0.0  0.0     1798.0  761.0

Std for State of New York has the highest number compared to Connecticut, Massachusetts, and New Jersey
The standard deviation measures how far apart the numbers are from the mean of the data set.
Interestingly, the average number of bedrooms is 3 with 2 bathrooms

In [158]:
states_df.columns

Index(['Unnamed: 0', 'price', 'bed', 'bath', 'acre_lot', 'house_size', 'state',
       'city'],
      dtype='object')

In [159]:
#New York Data in a df ny_cities_df
ny_cities_df = states_df[states_df['state'] == 'New York']
ny_cities_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
49777,54258,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,54530,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,54531,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,54539,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,54542,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...,...
63457,732127,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,732237,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,732292,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,732295,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [160]:
# #drop the column of the number which is the indexing row number from original raw data file /del the ['Unnamed:0'] from ny_cities_df
ny_cities_df = ny_cities_df[['price', 'bed', 'bath', 'acre_lot', 'house_size', 'state', 'city']]
ny_cities_df = ny_cities_df.dropna()
ny_cities_df

,price,bed,bath,acre_lot,house_size,state,city
49777,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...
63457,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [161]:
#get the count of how many cities are in the ny_cities_df
ny_cities= ny_cities_df.groupby('city')
ny_cities.count()

,price,bed,bath,acre_lot,house_size,state
city,,,,,,
Airmont,26,26,26,26,26,26
Amenia,14,14,14,14,14,14
Arden,5,5,5,5,5,5
Ardsley,22,22,22,22,22,22
Argyle,30,30,30,30,30,30
...,...,...,...,...,...,...
Witherbee,5,5,5,5,5,5
Woodhaven,27,27,27,27,27,27
Woodside,68,68,68,68,68,68


In [162]:
#find the mean in ny_cities df
ny_cities = ny_cities.mean().round()
ny_cities

C:\Users\Owner\AppData\Local\Temp\ipykernel_2856\273040110.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ny_cities = ny_cities.mean().round()


,price,bed,bath,acre_lot,house_size
city,,,,,
Airmont,830446.0,5.0,3.0,1.0,2363.0
Amenia,318500.0,3.0,2.0,1.0,1820.0
Arden,425000.0,4.0,3.0,0.0,1376.0
Ardsley,912591.0,4.0,3.0,0.0,2242.0
Argyle,226650.0,4.0,2.0,1.0,2183.0
...,...,...,...,...,...
Witherbee,59000.0,3.0,1.0,0.0,1425.0
Woodhaven,913667.0,5.0,3.0,0.0,1861.0
Woodside,1187170.0,4.0,2.0,0.0,1831.0


In [163]:
ny_cities.columns

Index(['price', 'bed', 'bath', 'acre_lot', 'house_size'], dtype='object')

In [164]:
#create columns for Lat and Lon.  Lat and Lon with emplty lists
ny_cities['Lat']=""
ny_cities['Lon']=""
ny_cities

,price,bed,bath,acre_lot,house_size,Lat,Lon
city,,,,,,,
Airmont,830446.0,5.0,3.0,1.0,2363.0,,
Amenia,318500.0,3.0,2.0,1.0,1820.0,,
Arden,425000.0,4.0,3.0,0.0,1376.0,,
Ardsley,912591.0,4.0,3.0,0.0,2242.0,,
Argyle,226650.0,4.0,2.0,1.0,2183.0,,
...,...,...,...,...,...,...,...
Witherbee,59000.0,3.0,1.0,0.0,1425.0,,
Woodhaven,913667.0,5.0,3.0,0.0,1861.0,,
Woodside,1187170.0,4.0,2.0,0.0,1831.0,,


In [165]:
#drop columns so can keep just the price, Lat and Lon. The df is indexed with the city
ny_cities = ny_cities[['price','Lat','Lon']]
ny_cities = ny_cities.dropna()
ny_cities

,price,Lat,Lon
city,,,
Airmont,830446.0,,
Amenia,318500.0,,
Arden,425000.0,,
Ardsley,912591.0,,
Argyle,226650.0,,
...,...,...,...
Witherbee,59000.0,,
Woodhaven,913667.0,,
Woodside,1187170.0,,


In [166]:
ny_cities = ny_cities['price']
ny_cities = ny_cities.dropna()
ny_cities

city
Airmont       830446.0
Amenia        318500.0
Arden         425000.0
Ardsley       912591.0
Argyle        226650.0
               ...    
Witherbee      59000.0
Woodhaven     913667.0
Woodside     1187170.0
Yonkers       590730.0
Yulan         425000.0
Name: price, Length: 274, dtype: float64

In [167]:
#Connecticut data
ct_cities_df = states_df[states_df['state'] == 'Connecticut']
ct_cities_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
0,27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
1,27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
2,27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
3,27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
4,27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...,...
49772,464216,755000.0,4.0,3.0,1.60,2586.0,Connecticut,Stamford
49773,464217,799000.0,4.0,3.0,1.52,3154.0,Connecticut,Stamford
49774,464222,799000.0,4.0,4.0,1.19,3128.0,Connecticut,Stamford
49775,464224,699000.0,4.0,3.0,1.04,2156.0,Connecticut,Stamford


In [168]:
#Massachusetts data
ma_cities_df = states_df[states_df['state'] == 'Massachusetts']
ma_cities_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
63462,24231,180000.0,2.0,1.0,0.34,676.0,Massachusetts,Agawam
63463,24236,239900.0,3.0,1.0,0.46,1196.0,Massachusetts,Agawam
63464,24237,525000.0,3.0,3.0,0.45,2314.0,Massachusetts,Agawam
63465,24238,289900.0,3.0,2.0,0.36,1276.0,Massachusetts,Agawam
63466,24241,275000.0,4.0,2.0,0.11,1732.0,Massachusetts,Agawam
...,...,...,...,...,...,...,...,...
134457,460678,409000.0,3.0,2.0,1.25,1792.0,Massachusetts,Otis
134458,460694,709000.0,2.0,2.0,0.37,1952.0,Massachusetts,Otis
134459,460714,175000.0,4.0,1.0,0.13,1554.0,Massachusetts,Great Barrington
134460,460716,159900.0,2.0,1.0,0.06,800.0,Massachusetts,Otis


In [169]:
#New Jersey data
nj_cities_df = states_df[states_df['state'] == 'New Jersey']
nj_cities_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
134462,30126,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
134463,45320,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
134464,385082,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
134465,388797,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
134466,389915,333490.0,3.0,3.0,0.07,1500.0,New Jersey,Burlington
...,...,...,...,...,...,...,...,...
185883,733133,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
185884,733136,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
185885,733146,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
185886,733147,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [170]:
#set index as state
states_df= states_df.set_index ("state")
states_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,city
state,,,,,,,
Connecticut,27821,579900.0,4.0,3.0,2.34,3320.0,Coventry
Connecticut,27826,215000.0,2.0,1.0,0.91,960.0,East Windsor
Connecticut,27828,144900.0,2.0,1.0,0.36,860.0,Vernon
Connecticut,27829,174900.0,3.0,2.0,0.19,1280.0,Manchester
Connecticut,27830,289900.0,4.0,2.0,0.38,1617.0,East Windsor
...,...,...,...,...,...,...,...
New Jersey,733133,475000.0,4.0,3.0,0.23,2252.0,South Brunswick
New Jersey,733136,350000.0,2.0,2.0,0.01,1551.0,Piscataway
New Jersey,733146,241000.0,2.0,2.0,0.50,1077.0,North Brunswick
